In [87]:

import joblib
import tensorflow as tf

clf, le, scaler, pca = joblib.load('CNN_Model.pkl')
vgg_model = tf.keras.models.load_model('vgg_model.h5')


In [88]:
vgg_model.inputs, vgg_model.outputs

([<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'zero_padding2d_13_input')>],
 [<KerasTensor: shape=(None, 2622) dtype=float32 (created by layer 'flatten_1')>])

In [89]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_eye.xml')

faces = face_cascade.detectMultiScale(gray, 1.3, 5)
faces

array([[357, 394, 568, 568]])

In [90]:
import cv2
def load_image(path):
    img = cv2.imread(path, 1)
    # OpenCV loads images with color channels
    # in BGR order. So we need to reverse them
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray)
        if len(eyes) >= 2:
            return roi_color[...,::-1]
    # return img[...,::-1]

In [91]:
import numpy as np

img_path = 'dwayne_johnson.jpg'

img = load_image(img_path)

# Normalising pixel values from [0-255] to [0-1]: scale RGB values to interval [0,1]
img = (img / 255.).astype(np.float32)
img = cv2.resize(img, dsize = (224,224))
print(img.shape)

# Obtain embedding vector for an image
# Get the embedding vector for the above image using vgg_face_descriptor model and print the shape 
embedding_vector = vgg_model.predict(np.expand_dims(img, axis=0))[0]
print(embedding_vector.shape)

(224, 224, 3)
1/1 [==============================] - 0s 270ms/step
(2622,)


In [92]:
embeddings = np.zeros((1,2622))
embeddings[0] = embedding_vector
x_inp = embeddings
x_inp_scaler = scaler.transform(x_inp)
x_inp_pca = pca.transform(x_inp_scaler)

x_inp_pca.shape

(1, 128)

In [93]:
y_predict = clf.predict(x_inp_pca)
y_predict


array([22], dtype=int64)

In [94]:
y_predict_decoded = le.inverse_transform(y_predict)
y_predict_decoded

array(['pins_Dwayne Johnson'], dtype='<U27')